# Powerplant Optimization with Differential Evolution

All information is taken from "Differential Evolution: A Survey of the State-of-the-Art"by Swagatam Das, Member, IEEE, and Ponnuthurai Nagaratnam Suganthan, Senior Member, IEEE.

In [1]:
import numpy as np

## 1. Initialization 

## 2. Mutation

## 3. Crossover / Recombination

## 4. Selection

## 5. Action

## 6. Plot